In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import requests as r
from Config2 import gkey
import json

city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()
city_data_df = city_data_df.set_index('City ID')

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,bambous virieux,MU,-20.3428,57.7575,74.21,88,75,3.44,broken clouds
1,1,bredasdorp,ZA,-34.5322,20.0403,60.12,82,0,2.30,clear sky
2,2,dingle,PH,10.9995,122.6711,83.26,76,88,8.14,overcast clouds
3,3,igrim,RU,63.1906,64.4162,-0.54,98,82,5.08,broken clouds
4,4,qaanaaq,GL,77.4840,-69.3632,-10.19,71,30,7.70,scattered clouds


In [ ]:
min_temp = float(input('What is your minimum tolerable temperature?'))
max_temp = float(input('What is your maximum tolerable temperature?'))

In [ ]:
preferred_temp_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp)&
                                           (city_data_df['Max Temp'] <= max_temp)]


In [ ]:
preferred_temp_cities_df.head()

In [ ]:
preferred_temp_cities_df.isnull().any()

In [ ]:
preferred_temp_cities_df = preferred_temp_cities_df.dropna()
preferred_temp_cities_df.isnull().any()
preferred_temp_cities_df.count()

In [3]:
import gmaps
gmaps.configure(api_key=gkey)

In [ ]:
hotel_df = preferred_temp_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng', 'Current Description']].copy()
hotel_df['Hotel Name'] = ''
hotel_df = hotel_df.drop_duplicates('City')
hotel_df
payload = {"radius": 5000, "type": "lodging", "key": gkey}
# base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
# hotel = r.get(base_url, params=payload).json()
# print(json.dumps(hotel, indent=4, sort_keys=True))


In [ ]:
for i, row in hotel_df.iterrows():
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    location = f'{lat},{lng}'
    payload['location'] = location
    hotel = r.get(base_url, params=payload).json()
    print(f'Getting data for {city}')
    try:
        hotel_df.loc[i,'Hotel Name'] = hotel['results'][0]['name']
    except IndexError:
        print('No hotel nearby, moving on...')
        pass
    

In [ ]:
hotel_df
hotel_df['City'] = hotel_df['City'].str.capitalize()
hotel_df.isnull().any()

In [ ]:
#code refactored from https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python
hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(how='any', inplace=True)

In [ ]:
hotel_df

In [ ]:
hotel_df.to_csv('WeatherPy_vacation.csv')

In [5]:
hotel_df = pd.read_csv('WeatherPy_vacation.csv')
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
fig = gmaps.figure()
marker_locations = hotel_df[['Lat', 'Lng']]
hotel_info = [info_box_template.format(**row) for i, row in hotel_df.iterrows()]
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))